# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [303]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

pd.set_option('max_colwidth', 100)

In [271]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperat...,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperat...,direct
4,12,"says: west side of Haiti, rest of the country today and tonight",facade ouest d Haiti et le reste du pays aujourd hui et ce soir,direct


In [272]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;...
1,7,related-1;request-0;offer-0;aid_related-1;medical_help-0;medical_products-0;search_and_rescue-0;...
2,8,related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;...
3,9,related-1;request-1;offer-0;aid_related-1;medical_help-0;medical_products-1;search_and_rescue-0;...
4,12,related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;...


In [273]:
# Get some understanding of datasets
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
id          26248 non-null int64
message     26248 non-null object
original    10184 non-null object
genre       26248 non-null object
dtypes: int64(1), object(3)
memory usage: 820.3+ KB


In [274]:
# NA values
messages.isna().sum()

id              0
message         0
original    16064
genre           0
dtype: int64

In [275]:
# Counts of unique values
for col in messages.columns:
    print(messages[col].value_counts())
    print('*********************')

24779    3
7747     2
14246    2
25512    2
17553    2
13914    2
29119    2
14135    2
14592    2
17919    2
18313    2
6573     2
18925    2
14073    2
12589    2
6327     2
27768    2
23002    2
6492     2
15938    2
2446     2
4956     2
1652     2
7892     2
11503    2
15576    2
3250     2
5776     2
6515     2
24247    2
        ..
27384    1
25337    1
29435    1
19196    1
8913     1
10960    1
21183    1
23230    1
14994    1
12947    1
2708     1
661      1
6806     1
4759     1
27288    1
25241    1
19100    1
17053    1
23198    1
21151    1
15026    1
12979    1
2740     1
6838     1
4791     1
27320    1
25273    1
29371    1
17085    1
2049     1
Name: id, Length: 26180, dtype: int64
*********************
#NAME?                                                                                                                                                                                                                                                                       

In [276]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
id            26248 non-null int64
categories    26248 non-null object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB


In [277]:
categories.isna().sum()

id            0
categories    0
dtype: int64

In [278]:
# Count unique values for categories dataset
for col in categories:
    print(categories[col].value_counts())

24779    3
7747     2
14246    2
25512    2
17553    2
13914    2
29119    2
14135    2
14592    2
17919    2
18313    2
6573     2
18925    2
14073    2
12589    2
6327     2
27768    2
23002    2
6492     2
15938    2
2446     2
4956     2
1652     2
7892     2
11503    2
15576    2
3250     2
5776     2
6515     2
24247    2
        ..
27384    1
25337    1
29435    1
19196    1
8913     1
10960    1
21183    1
23230    1
14994    1
12947    1
2708     1
661      1
6806     1
4759     1
27288    1
25241    1
19100    1
17053    1
23198    1
21151    1
15026    1
12979    1
2740     1
6838     1
4791     1
27320    1
25273    1
29371    1
17085    1
2049     1
Name: id, Length: 26180, dtype: int64
related-0;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tool

In [279]:
# Check similar records in both datasets
len(set(list(categories['id'])).intersection(list(messages['id'])))

26180

### 2. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [280]:
# create a dataframe of the 36 individual category columns
categories_split = categories['categories'].str.split(';', expand = True)
categories_split.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [281]:
# select the first row of the categories dataframe
row = categories_split.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x: x[:-2])
print(category_colnames)

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [282]:
categories_split.columns = category_colnames

### 3. Convert `categories_split` values to just numbers 0 or 1 and merge it with `categories` dataset
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [283]:
for column in categories_split:
    # set each value to be the last character of the string
    categories_split[column] = categories_split[column].apply(lambda x: x[-1])
    
    # convert column from string to numeric
    categories_split[column] = categories_split[column].astype(np.int64)
categories_split.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [284]:
categories.shape

(26248, 2)

In [285]:
categories = categories.drop('categories', axis=1)

In [286]:
# Merge the categories dataset with categories_split
categories_clean = pd.merge(categories,categories_split,left_index=True,right_index=True)
categories_clean.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4. Merge datasets.
- Merge the messages and clean categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [287]:
# merge datasets
df = pd.merge(messages,categories_clean,on = 'id', copy=False)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperat...,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperat...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country today and tonight",facade ouest d Haiti et le reste du pays aujourd hui et ce soir,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [288]:
df.shape

(26386, 40)

In [289]:
df['id'].value_counts()

24779    9
7747     4
14246    4
25512    4
17553    4
13914    4
29119    4
14135    4
14592    4
17919    4
18313    4
6573     4
18925    4
14073    4
12589    4
6327     4
27768    4
23002    4
6492     4
15938    4
2446     4
4956     4
1652     4
7892     4
11503    4
15576    4
3250     4
5776     4
6515     4
24247    4
        ..
27384    1
25337    1
29435    1
19196    1
8913     1
10960    1
21183    1
23230    1
14994    1
12947    1
2708     1
661      1
6806     1
4759     1
27288    1
25241    1
19100    1
17053    1
23198    1
21151    1
15026    1
12979    1
2740     1
6838     1
4791     1
27320    1
25273    1
29371    1
17085    1
2049     1
Name: id, Length: 26180, dtype: int64

In [290]:
len(set(list(df['id'])))

26180

In [291]:
# Checking NA values after merging
df.isna().sum()

id                            0
message                       0
original                  16140
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [292]:
df_count_unique = df['id'].value_counts()

In [293]:
# There are 273 duplicate IDs
df_count_unique[df_count_unique>=2].sum()

273

In [294]:
df.set_index('id').loc[df_count_unique[df_count_unique>=2].index]

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24779,"Shelter materials (thick polyesters) are being distributed to 18,000 households.",NaN,news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
df.duplicated(subset='id').sum()

206

### 5. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [296]:
# check number of duplicates
len(df)-len(df.drop_duplicates(subset='id'))

206

In [297]:
# drop duplicates
df_clean = df.drop_duplicates(subset='id')

In [298]:
# Checking any NA values apart from Original
df_clean.isna().sum()

id                            0
message                       0
original                  16027
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [302]:
# Confirm any duplicates
df_clean.duplicated(subset='id').sum()

0

In [300]:
df_clean.shape

(26180, 40)

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [304]:
engine = create_engine('sqlite:///DisasterResponse.db')
df_clean.to_sql('Response', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.